In [23]:
import pandas as pd
import numpy as np
import pylab 
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import os, sys

In [24]:
def subcategory_subscription_week_month_churn_rate(filename):



    # ### Instruction
    # Read your subset data and subscribers list (may change a little bit later) here and you will get two csv files in the same directory for weekly and monthly churn rate. It's the only place you need to do mannually. 

    # In[179]:


    # You need to change yourself
    subcategory_name = os.path.splitext(filename)[0]   

    # In[5]:
    df_sales = pd.read_csv("../data/subcategory/"+ subcategory_name + ".csv")


    # In[51]:

    df_subsribers = pd.read_csv('../data/subscribers_final_sheet.csv')



# ### Instruction
# Read your subset data and subscribers list (may change a little bit later) here and you will get two csv files in the same directory for weekly and monthly churn rate. It's the only place you need to do mannually. 

# In[2]:


    df_sales_date = df_sales
    df_sales_date['date']=pd.to_datetime(df_sales_date['order_date']).dt.strftime("%Y-%m-%d")
    df_sales_date=df_sales_date.sort_values(by=['date','address_hash']) 


    # In[6]:


    df_sales_171819 = df_sales_date.loc[(df_sales_date['date'] >= '2017-01-01') & (df_sales_date['date']!='2018-12-12')]


    # In[7]:


    # Now get subscribers from the subset.
    df_info_subscribers = df_subsribers[df_subsribers.set_index(['Ship-to Address Hash'])                    .index.isin(df_sales_171819.set_index(['address_hash']).index)]


    # ### Build the start_end table

    # In[8]:


    df_start_end = df_info_subscribers[['Ship-to Address Hash','first_pur','last_pur','avg_interval']]


    # In[9]:


    def f(x):    
        return pd.to_datetime(x['last_pur']) +  timedelta(days=int(x['avg_interval']))
    df_start_end['subs_churn'] = df_start_end.apply(f, axis=1)

    # for index, row in df_start_end.iterrows():
    #     row['subs_churn'] = pd.to_datetime(row['last_pur']) +  timedelta(days=int(row['avg_interval']))


    # In[10]:


    df_start_end_sort = df_start_end.sort_values(by=['first_pur','last_pur'])


    # In[11]:


    df_start_end_sort.reset_index(inplace = True)


    # In[12]:


    df_start_end_sort.drop(columns=['index'])


    # ## Churn Rates

    # **<center>Churn Rate = Customers Left / (New Customers + Existing Customers)</center>** <br/>
    # Left customers judgement: His end purchase date plus 3 months is earlier than the last day of the month. 3 months is the largest subsciption interval allowed. <br/>
    # New customers judgement: His start purchase date is later than the first day of the month. <br><br/>
    # First let's decide the day that marks the leave of each customer and append it to the df_sort we got earlier. In this part, we still stick to the whole customer base instead of the "subscribers" market. 

    # #### daily churn

    # Let's first create a new dataframe with each day from 2017-1-1 to 2020-2-10 as the key column 

    # In[13]:


    # Getting number of days need to fill the column
    from datetime import date
    d0 = date(2017, 1, 1)
    d1 = date(2020, 2, 10)
    delta = d1 - d0
    print(delta.days)


    # In[14]:


    # create an array of dates starting at '2017-01-01', one per day. and set day as index
    rng = pd.date_range('2017-01-01', periods=1135, freq='D')
    df_calender = pd.DataFrame({ 'Date': rng}) 


    # new_customer fill

    # In[15]:


    i = 0
    for index, row in df_calender.iterrows():
        count = 0
        while i < df_start_end_sort.shape[0] and pd.to_datetime(df_start_end_sort.at[i,'first_pur']) == pd.to_datetime(row['Date']):
            count += 1
            i += 1
            if i >= df_start_end_sort.shape[0]:
                break
        df_calender.at[index,'new_subs'] = count


    # In[16]:


    df_sort_by_date_end = df_start_end_sort.sort_values(by=['last_pur'])


    # In[17]:


    i = 0
    for index, row in df_calender.iterrows():
        count = 0
        while i < df_start_end_sort.shape[0] and pd.to_datetime(df_start_end_sort.at[i,'last_pur']) == pd.to_datetime(row['Date']):
            count += 1
            i += 1
            if i >= df_start_end_sort.shape[0]:
                break
        df_calender.at[index,'new_subs_churn'] = count


    # #### Now calculate the daily existing customer column
    # The first one or two months existing subscribers base may not be very accurate due to ignorance of active subscribers coming from the end of 2016. This error will gradually disappear after two months or so. Let's first regard the number of unique subscribers during the first month of 2017 as existing subscribers.

    # In[18]:


    df_sales_16_Dec = df_info_subscribers.loc[(df_info_subscribers['first_pur'] >= '2017-01-01')                                 & (df_info_subscribers['first_pur'] < '2017-02-01') ]


    # In[19]:


    existing_customer = df_sales_16_Dec['Ship-to Address Hash'].nunique()


    # In[20]:


    df_existing = df_calender


    # In[21]:


    df_existing.at[0,'existing_subscribers'] = existing_customer


    # In[22]:


    df_existing.at[0,'total_subscribers'] = existing_customer


    # In[23]:


    for i in range(1,df_existing.shape[0]):
        df_existing.at[i,'existing_subscribers'] = df_existing.at[i-1,'total_subscribers'] 
        df_existing.at[i,'total_subscribers'] = df_existing.at[i,'existing_subscribers']     + df_existing.at[i,'new_subs'] - df_existing.at[i,'new_subs_churn']


    # #### Monthly/Weekly churn

    # <img src="churn_rate.png" width=500 height=500 />
    # 
    # https://www.profitwell.com/blog/the-complete-saas-guide-to-calculating-churn-rate-and-keeping-it-simple

    # In[24]:


    df_month_week_existing = df_existing


    # In[25]:


    df_month_week_existing['Month'] = pd.to_datetime(df_month_week_existing['Date']).dt.to_period('M')


    # In[26]:


    df_month_week_existing['Date'] = pd.to_datetime(df_month_week_existing['Date'], errors='coerce')
    df_month_week_existing['Week'] = df_month_week_existing['Date'].dt.strftime('%Y-%U')
    #df.formatted_date.apply(lambda x: x.weekofyear)


    # ### Calculate monthly/weekly churn rate is a new and more precise way
    # <img src="shopify_way.png" width=500 height=500 />
    # https://www.profitwell.com/blog/the-complete-saas-guide-to-calculating-churn-rate-and-keeping-it-simple

    # In[27]:


    df_month_exist = df_month_week_existing.groupby(['Month']).agg({'new_subs_churn':'sum','existing_subscribers':'mean'})


    # In[28]:


    df_month_exist['monthly_subs_churn_rate'] = df_month_exist['new_subs_churn']/df_month_exist['existing_subscribers'] 


    # In[29]:


    df_month_exist = df_month_exist.fillna(0)


    # In[30]:


    df_week_exist = df_month_week_existing.groupby(['Week']).agg({'new_subs_churn':'sum','existing_subscribers':'mean'})


    # In[31]:


    df_week_exist['weekly_subs_churn_rate'] = df_week_exist['new_subs_churn']/df_week_exist['existing_subscribers'] 


    # In[32]:




    dfs = {subcategory_name + "_monthly_churn_rate":df_month_exist, subcategory_name + "_weekly_churn_rate":df_week_exist} 

    for k, v in dfs.items():
        path_name = '../output/subcategory_churn_rate/subscribers/' + subcategory_name + '_subs_churn/'
        if not os.path.exists(path_name):
            os.makedirs(path_name)
        v.to_csv(path_name + '{}.csv'.format(k))


In [25]:
for filename in os.listdir("../data/subcategory/"):
    subcategory_subscription_week_month_churn_rate(filename)
#     print(os.path.splitext(filename))
#     subcategory_subscription_week_month_churn_rate(filename)

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1135


/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1135


/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1135


/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1135


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series